In [8]:
import numpy as np
import helper

from cnn_model import cnn_model
from loss_functions import symmetric_cross_entropy
import tensorflow as tf
from tensorflow import keras

In [3]:
# FashionMNIST0.3.npz
Xtr, Str, Xts, Yts, T = helper.load_and_inspect("datasets/FashionMNIST0.3.npz", "FashionMNIST0.3")

Dataset loaded with the following shapes:
Xtr: (18000, 28, 28, 1)
Str: (18000,)
Xts: (3000, 28, 28, 1)
Yts: (3000,)


In [9]:
RANDOM_SEED = 42

In [ ]:
def train_model(X_train, y_train, X_val, y_val, method="fc", transition_matrix=None, epochs=50, input_shape=(28, 28, 1), num_classes=3):
    
    model = cnn_model(input_shape=input_shape, num_classes=num_classes)

    if method == "sce":
        loss_function = symmetric_cross_entropy(alpha=0.1, beta=1.0, num_classes=num_classes)

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.001),
        loss = loss_function,
        metrics=['accuracy']
    )

    early_stopping = keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True
    )

    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=128,
        callbacks=[early_stopping],
        verbose=0
    )

    return model

def evaluate_model(model, X_test, y_test):
    predictions = model.predict(X_test, verbose=0)
    predicted_classes = np.argmax(predictions, axis=1)
    accuracy = np.mean(predicted_classes == y_test) * 100
    return accuracy

def run_experiments(Xtr, Str, Xts, Yts, T, method, num_runs=10, epochs=50):
    input_shape = Xtr.shape[1:] 
    
    if method == 'fl':
        if T is not None:
            transition_matrix = T
        else:
            #call estimate T function here
            pass
    else:
        transition_matrix=None

    accuracies = []

    for run in range(num_runs):
        seed = RANDOM_SEED + run

        X_train, y_train, X_val, y_val = helper.split_data(
            Xtr, Str, train_ratio=0.8, random_seed=seed
        )

        model = train_model(X_train, y_train, X_val, y_val, method=method, transition_matrix=transition_matrix, epochs=epochs, input_shape=input_shape, num_classes=3)

        accuracy = evaluate_model(model, Xts, Yts)
        accuracies.append(accuracy)

        print(f"Run {run+1}/{num_runs}: Test Accuracy = {accuracy:.2f}%")

        tf.keras.backend.clear_session()
    
    mean_acc = np.mean(accuracies)
    std_acc = np.std(accuracies)
    
    print(f"Results for {method.upper()} method:")
    print(f"Mean Accuracy: {mean_acc:.2f} ± {std_acc:.2f}%")
    
    return accuracies
    

In [ ]:
run_experiments(Xtr, Str, Xts, Yts, T, method='sce', num_runs=10, epochs=10)